In [13]:
from numpy.random import seed
import os
import random
import numpy as np
import skimage
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras import backend as keras
from utils import DataGenerator
from unet import *
from main import showHistory, TrainValTensorBoard


In [14]:
params = {'batch_size':1,
          'dim':(128,128,128),
          'n_channels':1,
          'shuffle': True}
seismPathT = "../data_seismic/train/seis/"
faultPathT = "../data_seismic/train/fault/"
seismPathV = "../data_seismic/validation/seis/"
faultPathV = "../data_seismic/validation/fault/"
train_ID = range(200)
valid_ID = range(20)
train_generator = DataGenerator(dpath=seismPathT,fpath=faultPathT,
                                data_IDs=train_ID,**params)
valid_generator = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                data_IDs=valid_ID,**params)
model = unet(input_size=(None, None, None,1))
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d_45 (Conv3D)             (None, None, None,   448         ['input_4[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_46 (Conv3D)             (None, None, None,   6928        ['conv3d_45[0][0]']              
                                None, 16)                                                   

In [15]:
# checkpoint
filepath="check1/fseg-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
      verbose=1, save_best_only=False, mode='max')
logging = TrainValTensorBoard()
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
#                              patience=20, min_lr=1e-8)
callbacks_list = [checkpoint, logging]
print("data prepared, ready to train!")
# Fit the model
history=model.fit_generator(generator=train_generator,
validation_data=valid_generator,epochs=100,callbacks=callbacks_list,verbose=1)
model.save('check1/fseg.hdf5')
showHistory(history)

data prepared, ready to train!


/tmp/ipykernel_57846/2287510817.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=train_generator,


Epoch 1/100
200/200 [==============================] - ETA: 0s - loss: 0.2824 - accuracy: 0.9232
Epoch 1: saving model to check1/fseg-01.hdf5


AttributeError: 'function' object has no attribute 'eval'